
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 複数の入力チャンネルと複数の出力チャンネル

:label: `sec_channels`

これまで、各画像を構成する複数のチャネル (たとえば、カラー画像には赤、緑、青の量を示す標準 RGB チャネルがあります) と複数のチャネルの畳み込み層について :numref: `subsec_why-conv-channels`で説明しましたが、単一の入力チャネルと単一の出力チャネルだけを使用して、すべての数値例を簡略化しました。これにより、入力、畳み込みカーネル、出力をそれぞれ 2 次元のテンソルとして考えることができました。

チャンネルをミックスに追加すると、入力と隠れた表現の両方が 3 次元テンソルになります。たとえば、各 RGB 入力イメージの形状は $3\times h\times w$ です。サイズ 3 のこの軸を*チャネルの*寸法と呼びます。チャネルの概念は、CNN 自体と同じくらい古いものです。たとえば、LeNet5 :cite: `LeCun.Jackel.Bottou.ea.1995`はそれらを使用します。このセクションでは、複数の入力チャネルと複数の出力チャネルを備えたコンボリューション カーネルについて詳しく見ていきます。


In [1]:
import torch
from d2l import torch as d2l


## 複数の入力チャンネル

入力データに複数のチャネルが含まれる場合、入力データとの相互相関を実行できるように、入力データと同じ数の入力チャネルを持つコンボリューション カーネルを構築する必要があります。入力データのチャネル数を $c_i$ とすると、コンボリューションカーネルの入力チャネル数も $c_i$ 必要になります。畳み込みカーネルのウィンドウ形状が $k_h\times k_w$ である場合、$c_i=1$ のとき、畳み込みカーネルは形状 $k_h\times k_w$ の単なる 2 次元テンソルと考えることができます。

ただし、$c_i&gt;1$ の場合、入力チャネル*ごと*に $k_h\times k_w$ の形状のテンソルを含むカーネルが必要になります。これらの $c_i$ テンソルを連結すると、形状 $c_i\times k_h\times k_w$ の畳み込みカーネルが生成されます。入力カーネルと畳み込みカーネルにはそれぞれ $c_i$ チャネルがあるため、各チャネルの入力の 2 次元テンソルと畳み込みカーネルの 2 次元テンソルに対して相互相関演算を実行し、$c_i$ の結果を加算できます。一緒に (チャネル全体で合計して) 2 次元テンソルを生成します。これは、複数チャネル入力と複数入力チャネル コンボリューション カーネルの間の 2 次元相互相関の結果です。

 :numref: `fig_conv_multi_in` 2 つの入力チャネルを使用した 2 次元相互相関の例を示します。影付きの部分は、最初の出力要素と、出力の計算に使用される入力およびカーネル テンソル要素です: $(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1) +3\times2+4\times3)=56$。 

![](../img/conv-multi-in.svg) :label: `fig_conv_multi_in`

ここで何が起こっているのかを本当に理解するために、私たち自身で（**複数の入力チャネルを使用して相互相関演算を実装する**）ことができます。ここで行っているのは、チャネルごとに相互相関演算を実行し、その結果を合計していることだけであることに注意してください。


In [2]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))


:numref: `fig_conv_multi_in`の値に対応する入力テンソル`X`とカーネル テンソル`K`を構築して、相互相関演算の (**出力を検証する**) ことができます。


In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])


## 複数の出力チャンネル

:label: `subsec_multi-output-channels`

入力チャンネルの数に関係なく、これまでのところ、出力チャンネルは常に 1 つになります。ただし、 :numref: `subsec_why-conv-channels`で説明したように、各層に複数のチャネルを持つことが不可欠であることがわかります。最も一般的なニューラル ネットワーク アーキテクチャでは、ニューラル ネットワークを深くするにつれて実際にチャネル次元を増加させます。通常、*チャネル深度*を高めるために空間解像度をトレードオフするためにダウンサンプリングが行われます。直観的には、各チャネルが異なる機能セットに応答すると考えることができます。現実はこれよりもう少し複雑です。素朴な解釈では、表現はピクセルごとまたはチャネルごとに独立して学習されると考えられます。代わりに、チャネルは共同で役立つように最適化されています。これは、単一のチャネルをエッジ検出器にマッピングするのではなく、単にチャネル空間内の特定の方向がエッジの検出に対応することを意味する可能性があることを意味します。

 $c_i$ と $c_o$ はそれぞれ入力チャネルと出力チャネルの数を表し、$k_h$ と $k_w$ をカーネルの高さと幅とします。複数のチャネルを持つ出力を取得するには、出力チャネル*ごと*に $c_i\times k_h\times k_w$ の形状のカーネル テンソルを作成できます。それらを出力チャネル次元で連結し、畳み込みカーネルの形状が $c_o\times c_i\times k_h\times k_w$ になるようにします。相互相関演算では、各出力チャネルの結果は、その出力チャネルに対応する畳み込みカーネルから計算され、入力テンソルのすべてのチャネルから入力を受け取ります。

以下に示すように [**複数のチャネルの出力を計算する**] ための相互相関関数を実装します。


In [4]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)


`K`のカーネル テンソルを`K+1`および`K+2`と連結することにより、3 つの出力チャネルを持つ自明な畳み込みカーネルを構築します。


In [5]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])


以下では、入力テンソル`X`に対してカーネル テンソル`K`を使用して相互相関演算を実行します。これで、出力には 3 つのチャンネルが含まれます。最初のチャネルの結果は、前の入力テンソル`X`および複数入力チャネル、単一出力チャネル カーネルの結果と一致します。


In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


## $1\times 1$ 畳み込み層

:label: `subsec_1x1`

最初は、[ **$1 \times 1$ 畳み込み**]、つまり $k_h = k_w = 1$ はあまり意味がないようです。結局のところ、畳み込みは隣接するピクセルを相関させます。 $1 \times 1$ 畳み込みは明らかにそうではありません。それにもかかわらず、これらは複雑なディープ ネットワークの設計に組み込まれることがある一般的な操作です :cite: `Lin.Chen.Yan.2013,Szegedy.Ioffe.Vanhoucke.ea.2017`実際にどのような動作をするかを詳しく見てみましょう。

最小ウィンドウが使用されるため、$1\times 1$ 畳み込みでは、高さと幅の次元で隣接する要素間の相互作用からなるパターンを認識する、より大きな畳み込み層の機能が失われます。 $1\times 1$ 畳み込みの唯一の計算はチャネル次元で発生します。

 :numref: `fig_conv_1x1` 、3 つの入力チャネルと 2 つの出力チャネルを持つ $1\times 1$ 畳み込みカーネルを使用した相互相関計算を示します。入力と出力の高さと幅が同じであることに注意してください。出力内の各要素は、入力画像内の*同じ位置にある*要素の線形結合から導出されます。 $1\times 1$ 畳み込み層は、$c_i$ に対応する入力値を $c_o$ 出力値に変換するために、すべての単一ピクセル位置に適用される全結合層を構成すると考えることができます。これは依然として畳み込み層であるため、重みはピクセル位置全体で結び付けられます。したがって、$1\times 1$ 畳み込み層には $c_o\times c_i$ の重み (プラスバイアス) が必要です。また、畳み込み層の後には通常、非線形性が伴うことにも注意してください。これにより、$1 \times 1$ 畳み込みを他の畳み込みに単純に折り畳むことができなくなります。 

![](../img/conv-1x1.svg) :ラベル: `fig_conv_1x1`

これが実際に機能するかどうかを確認してみましょう。全結合層を使用して $1 \times 1$ 畳み込みを実装します。唯一のことは、行列の乗算の前後でデータの形状を調整する必要があるということです。


In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


$1\times 1$ 畳み込みを実行する場合、上記の関数は、以前に実装された相互相関関数`corr2d_multi_in_out`と同等です。いくつかのサンプルデータを使用してこれを確認してみましょう。


In [8]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6


## 議論

チャネルを使用すると、重大な非線形性を可能にする MLP と、特徴の*局所的な*分析を可能にする畳み込みという、両方の長所を組み合わせることができます。特に、チャネルを使用すると、CNN はエッジ検出器や形状検出器などの複数の特徴を同時に推論できるようになります。また、翻訳の不変性と局所性から生じるパラメータの大幅な削減と、コンピュータ ビジョンにおける表現力豊かで多様なモデルの必要性との間の実際的なトレードオフも提供します。

ただし、この柔軟性には代償が伴うことに注意してください。サイズ $(h \times w)$ の画像の場合、 $k \times k$ 畳み込みを計算するコストは $\mathcal{O}(h \cdot w \cdot k^2)$ です。 $c_i$ と $c_o$ の入力チャネルと出力チャネルの場合、これはそれぞれ $\mathcal{O}(h \cdot w \cdot k^2 \cdot c_i \cdot c_o)$ に増加します。 $5 \times 5$ のカーネルと $128$ の入出力チャネルを備えた $256 \times 256$ のピクセル イメージの場合、これは 530 億回を超える演算に相当します (乗算と加算は別々に数えます)。後で、コストを削減するための効果的な戦略に遭遇します。たとえば、チャネルごとの操作をブロック対角にすることを要求し、 ResNeXt :cite: `Xie.Girshick.Dollar.ea.2017`のようなアーキテクチャにつながります。

## 演習
1. サイズがそれぞれ $k_1$ と $k_2$ の 2 つの畳み込みカーネルがあると仮定します (間に非線形性はありません)。<ol><li>演算の結果が単一の畳み込みで表現できることを証明します。
1. 等価な単一の畳み込みの次元は何ですか?
1. 逆は真ですか?つまり、畳み込みを常に 2 つの小さな畳み込みに分解できますか?
1. 順伝播の計算コスト (乗算と加算) はいくらですか?
1. メモリ使用量とは何ですか?
1. 逆方向計算のメモリ使用量はどれくらいですか?
1. バックプロパゲーションの計算コストは​​いくらですか?
1. 行列が $b$ ブロックに分割されている場合、ブロック対角行列の乗算はどれくらい速くなりますか?
1.  $b$ ブロックがあることの欠点は何ですか?少なくとも部分的にはどうすれば修正できるでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/70)
